# 나만의 여행 가이드 보이스 챗봇

### 사용자 입력을 음성으로 받는다.

### STT를 적용하여 텍스트로 변환한다.

### 변환된 텍스트를 입력으로 하여 프롬프트 엔지니어링을 해 API 요청을 보낸다. 

### 반환받은 응답을 TTS를 적용하여 음성으로 재생한다. 

### (+) 2에서 입력된 텍스트와 4에서 반환된 응답을 채팅 내역 보듯이 (카카오톡 대화처럼) 화면에 출력한다.

In [50]:
from dotenv import load_dotenv
import os

load_dotenv()
OPEN_API_KEY = os.getenv('OPEN_API_KEY')

In [51]:
import speech_recognition as sr

recognizer = sr.Recognizer()

with sr.Microphone() as source:
    print("말씀하세요...")
    audio = recognizer.listen(source)  
    with open("tts_output_voice.mp3", "wb") as f:
        f.write(audio.get_wav_data())   

말씀하세요...


In [52]:
from openai import OpenAI
client = OpenAI()

with open("tts_output_voice.mp3", "rb") as f:
    transcriptions = client.audio.transcriptions.create(
        model="whisper-1",
        file=f,
        language="ko"
    )

query_text = transcriptions.text
print("음성 인식 결과:", query_text)

음성 인식 결과: 피자의 나라 이탈리아


In [ ]:
from openai import OpenAI
import json

def my_travel_guide(query: str, temperature: float = 0.5, model: str = "gpt-4o-mini"):
    """
    '나만의 여행 가이드 보이스 챗봇'
    사용자의 여행 조건(query)에 맞춰 맞춤 여행 코스/팁을 JSON으로 반환한다.
    """
    client = OpenAI()

    system_instruction = """
너는 친절한 '개인 여행 가이드'이다. 한국어로 말한다.
사용자가 제공한 여행 조건(도시, 기간, 동행자, 예산, 관심사 등)을 분석하여
실행 가능한 맞춤 여행 일정을 제안한다.

[응답 원칙]
- 하루 단위 또는 시간대별 동선으로 추천 코스를 정리한다.
- 유명 관광지 + 숨은 명소를 균형 있게 제안한다.
- 각 관광지/활동에 대해 간단한 설명뿐 아니라 추가적인 배경·특징 설명을 제공한다.
- 교통(대중교통/도보/렌트카)과 티켓 팁(예매, 휴무일) 포함한다.
- 날씨/혼잡도 주의사항 반영한다.
- 비/휴무 대비 대체 플랜 반드시 포함한다.
- 말투는 친근하고 간결하게 표현한다.

[출력(JSON)]
{
  "trip_summary": "한 줄 요약",
  "itinerary": [
    {
      "day": 1,
      "schedule": [
        {
          "time": "오전",
          "place": "관광지/활동",
          "note": "간단한 설명",
          "description": "관광지의 역사, 특징, 방문 포인트 등 추가 설명"
        },
        {
          "time": "오후",
          "place": "관광지/활동",
          "note": "간단한 설명",
          "description": "관광지의 역사, 특징, 방문 포인트 등 추가 설명"
        }
      ]
    }
  ],
  "food_recommendations": [
    {"name": "맛집/카페", "comment": "추천 이유"}
  ],
  "transport_tips": ["교통/티켓 관련 팁"],
  "weather_notes": ["날씨/혼잡도 주의사항"],
  "backup_plan": ["비/휴무 대비 대체 일정"]
}
"""

    user_message = f"""
[여행 조건/상황]
{query}

조건이 부족하면 도시, 기간, 동행자, 예산, 관심사를 합리적으로 보완해준다.
JSON 스키마의 키는 반드시 그대로 유지한다.
"""

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_instruction},
            {"role": "user", "content": user_message},
        ],
        response_format={"type": "json_object"},
        temperature=temperature,
        max_tokens=1500,
    )

    content = resp.choices[0].message.content
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        return {"raw_text": content}


In [54]:
result = my_travel_guide(query_text)

print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "trip_summary": "이탈리아에서의 5일간의 피자 탐방 여행",
  "itinerary": [
    {
      "day": 1,
      "schedule": [
        {
          "time": "오전",
          "place": "로마 콜로세움",
          "note": "고대 로마의 상징",
          "description": "로마의 대표적인 관광지로, 고대 로마의 검투사 경기가 열리던 곳입니다. 역사적인 건축물과 함께 주변의 포로 로마노도 함께 둘러보세요."
        },
        {
          "time": "오후",
          "place": "트라스테베레",
          "note": "로마의 매력적인 동네",
          "description": "이탈리아 전통의 피자를 맛볼 수 있는 숨겨진 명소입니다. 아기자기한 골목길과 현지 분위기를 만끽할 수 있습니다."
        }
      ]
    },
    {
      "day": 2,
      "schedule": [
        {
          "time": "오전",
          "place": "바티칸 시국",
          "note": "세계에서 가장 작은 국가",
          "description": "성 베드로 대성당과 시스티나 성당을 방문하여 미켈란젤로의 작품을 감상하세요. 입장 전에 미리 예약하는 것이 좋습니다."
        },
        {
          "time": "오후",
          "place": "피자 마르게리타 맛집",
          "note": "전통 피자의 원조",
          "description": "나폴리 스타일의 피자를 즐길 수 있는 유명 맛집으로, 신선한 재료와 전통 방식으로 만든 피자를 경험하세요."
        }
      ]
    },
    {
      "day": 3,


In [55]:
import json
from openai import OpenAI

result = my_travel_guide(query_text)

print(f"[사용자] {query_text}")
print("[가이드] 여행 일정을 정리해드릴게요!\n")

for day in result["itinerary"]:
    print(f"{day['day']}일차 일정")
    for sch in day["schedule"]:
        print(f"{sch['time']} - {sch['place']} ({sch['note']})")
        print(f"{sch['description']}")
    print()


tts_input = json.dumps(result, ensure_ascii=False, indent=2)

client = OpenAI()

with client.audio.speech.with_streaming_response.create(
    model="tts-1",
    voice="nova",
    input=tts_input
) as response:
    response.stream_to_file("guide_reply.mp3")


[사용자] 피자의 나라 이탈리아
[가이드] 여행 일정을 정리해드릴게요!

1일차 일정
오전 - 콜로세움 (고대 로마의 상징)
콜로세움은 고대 로마 시대의 원형 경기장으로, 로마의 역사와 건축을 느낄 수 있는 곳입니다. 오전 일찍 방문하면 혼잡함을 피할 수 있습니다.
오후 - 로마 포럼 (고대 로마의 중심지)
로마 포럼은 고대 로마의 정치, 상업, 종교의 중심지였습니다. 다양한 유적들을 감상하며 로마의 역사를 배울 수 있습니다.

2일차 일정
오전 - 바티칸 시국 (세계에서 가장 작은 국가)
바티칸은 교황의 거주지로, 성 베드로 대성당과 시스티나 성당이 유명합니다. 미리 예약하면 대기 시간을 줄일 수 있습니다.
오후 - 트라스테베레 (로마의 숨은 명소)
트라스테베레는 아기자기한 골목과 전통적인 이탈리안 레스토랑이 있는 지역으로, 현지 분위기를 만끽할 수 있습니다.

3일차 일정
오전 - 산타 마리아 마조레 성당 (아름다운 모자이크)
산타 마리아 마조레 성당은 로마에서 가장 큰 성당 중 하나로, 화려한 모자이크와 대리석 장식이 인상적입니다.
오후 - 피자 만들기 클래스 (이탈리아 전통 피자 체험)
로컬 셰프와 함께하는 피자 만들기 클래스에서 직접 피자를 만들어보며 이탈리아의 요리를 배워보세요.

4일차 일정
오전 - 판테온 (고대 로마의 신전)
판테온은 고대 로마의 신들에게 바쳐진 신전으로, 그 독특한 돔 구조가 매력적입니다. 무료 입장이 가능합니다.
오후 - 스페인 계단 (로마의 인기 명소)
스페인 계단은 유명한 관광지로, 계단에 앉아 사람들을 구경하거나 사진을 찍기 좋은 장소입니다.



In [56]:
import os, json
from openai import OpenAI

HISTORY_FILE = "chat_history.json"

# 0) 실행 시작 시 기존 대화 불러오기
if os.path.exists(HISTORY_FILE):
    with open(HISTORY_FILE, "r", encoding="utf-8") as f:
        chat_history = json.load(f)
else:
    chat_history = []

client = OpenAI()

# 1) STT로 얻은 텍스트 (예: query_text)와 가이드 응답 만들기
result = my_travel_guide(query_text)

# 2) 채팅처럼 누적
chat_history.append(f"[사용자] {query_text}")

reply_lines = ["[가이드] 여행 일정을 정리해드릴게요!\n"]
for day in result["itinerary"]:
    reply_lines.append(f"{day['day']}일차 일정")
    for sch in day["schedule"]:
        reply_lines.append(f"{sch['time']} - {sch['place']} ({sch['note']})")
        reply_lines.append(sch["description"])
reply_text = "\n".join(reply_lines)

chat_history.append(reply_text)

# 3) 화면에 ‘누적’ 출력
for msg in chat_history:
    print("-" * 50)
    print(msg, end="\n\n")
print("-" * 50)

# 4) 파일에 저장(다음 실행 때 계속 이어보기)
with open(HISTORY_FILE, "w", encoding="utf-8") as f:
    json.dump(chat_history, f, ensure_ascii=False, indent=2)

--------------------------------------------------
[사용자] 삼척

--------------------------------------------------
[가이드] 여행 일정을 정리해드릴게요!

1일차 일정
오전 - 삼척 해변 (아름다운 바다와 해변 산책)
삼척 해변은 맑은 바다와 고운 모래사장이 특징입니다. 해변을 따라 산책하며 바다의 경치를 즐기고, 조개를 줍는 재미도 있습니다.
오후 - 삼척 대이리마을 (전통 마을 탐방)
대이리마을은 전통적인 한옥과 아름다운 자연경관이 어우러져 있습니다. 마을을 둘러보며 한국의 전통 문화를 느껴보세요. 특히, 지역 특산물인 한과를 맛보는 것도 추천합니다.
2일차 일정
오전 - 삼척 동굴 (신비로운 동굴 탐험)
삼척 동굴은 다양한 석회암 형성과 아름다운 조명으로 유명합니다. 동굴 내부를 탐험하며 자연의 신비로움을 느껴보세요. 입장 시 미끄러운 바닥에 주의하세요.
오후 - 오십천 강변 산책 (자연 속 힐링)
오십천 강변은 조용하고 평화로운 산책로가 있어, 여유롭게 자연을 즐기기에 좋습니다. 강변에 앉아 여유로운 시간을 가지며 사진도 찍어보세요.

--------------------------------------------------
[사용자] 시청해 주셔서 감사합니다.

--------------------------------------------------
[가이드] 여행 일정을 정리해드릴게요!

1일차 일정
오전 - 경복궁 (한국의 전통 궁궐)
경복궁은 조선 왕조의 주요 궁궐로, 아름다운 전통 건축과 정원이 매력적입니다. 궁궐 내에서의 가이드 투어를 통해 역사적인 이야기를 들으며 둘러보세요.
오후 - 북촌 한옥마을 (전통 한옥 마을)
북촌 한옥마을은 조선 시대의 전통 가옥들이 잘 보존되어 있는 곳으로, 고즈넉한 분위기 속에서 산책하며 한국의 전통 문화를 느낄 수 있습니다.
2일차 일정
오전 - 명동 (쇼핑과 길거리 음식)
명동은 서울의 대표적인 쇼핑 거리로, 다양한 브랜